In [6]:
import transformers, huggingface_hub, nltk, torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tarikrashada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Model

In [13]:
model_ckpt = 'facebook/bart-large-mnli'

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

def load_model(task,model_ckpt,device):
    pipe = transformers.pipeline(task=task,
                                model=model_ckpt,
                                device=device)
    return pipe

print(device)

mps


In [14]:
theme_classifier = load_model('zero-shot-classification',model_ckpt,device)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
theme_list = ['coming-of-age','journey','self development','']

## Process Dataset

In [44]:
subtitle_path = '../data/subtitles.txt'
import pandas as pd

def datasetProcessing(path):
    dialogue = []
    with open(path,'r') as file:
        lines = file.readlines()
        lines = lines[1:]
        
    for line in lines: 
        dialogue.append(line.split(',')[2])
    
    for i in range(len(dialogue)):
        dialogue[i] = dialogue[i].replace('"','')
        
    script = " ".join(dialogue)
    
    return script

script = datasetProcessing(subtitle_path)

In [55]:
def getThemeOutputs(script,theme_list,pipeline):
    
    script_sentences = nltk.sent_tokenize(script)
    batch_size = 20
    script_batches = []
    
    for index in range(0, len(script_sentences), batch_size):
        sentences = " ".join(script_sentences[index:index + batch_size])
        script_batches.append(sentences)
        
    themes = {}
    outputs = pipeline(script_batches,theme_list,multi_label=True)
    for output in outputs:
        for label, score in zip(output['labels'],output['scores']):
            if label not in themes:
                themes[label] = []
            themes[label].append(score)
    
    themes = {key : np.mean(np.array(value)) for key,value in themes.items()}
    return themes 
            